# Алгоритм Баума - Велша

Алгоритм для нахождения неизвестных параметров скрытой марковской модели. Он использует алгоритм прямого-обратного хода и является частным случаем обобщённого EM-алгоритма.

## Описание

Исходные данные: λ = (A, B, π) со случайными начальными условиями. 

Пусть Qt  — это дискретная случайная переменная, принимающая одно из N значений (1 … N). Будем полагать, что данная модель Маркова, определенная как P(Qt∣Qt−1) независима от t. Тогда можно задать P(Qt∣Qt−1) как независящую от времени стохастическую матрицу перемещений $$A=\left\{a_{ij}\right\}=\rho\left( Q_{t}=j \mid Q_{t-1} = i \right)$$ Особый случай для времени t=1 определяется начальным распределением $$\pi_{i}=P\left(Q_{1}=i\right)$$.

Будем считать, что мы в состоянии j в момент времени t, если Qt=j. Последовательность заданных состояний определяется как q={q1…qT}, где qt ∈ {1…N} является состоянием в момент времени t.

Наблюдение может иметь одно из L возможных значений, Qt ∈ {o1…oL}. Вероятность заданного вектора наблюдений в момент времени t для состояния j определяется как $$b_{j}\left(o_{t}\right) = P\left( O_{t} = o_{t} \mid Q_{t} = j\right) \left(B=\left\{b_{ij}\right\} - это\: матрица\: L\: на\: N\right)$$ Заданная последовательность наблюдений O выражается как
$$O = \left(O_{1}=o_{1}, \: ...\:, O_{T}=o_{T}\right)$$

Следовательно, мы можем описать скрытую модель Маркова с помощью λ=(A,B,π). При заданном векторе наблюдений O алгоритм Баума-Велша находит $$\lambda^{*} = \max_{\lambda} P\left(O\mid\lambda\right)$$  λ максимизирует вероятность наблюдений O.




## Реализация на python

In [5]:
import pandas as pd
import numpy as np

### Прямая процедура

In [6]:
def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]
 
    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1] @ a[:, j] * b[j, V[t]]
 
    return alpha

### Обратная процедура

In [7]:
def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))
 
    # beta от T = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
    # Обратный цикл от T-2 до 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]) @ a[j, :]
 
    return beta

### Алгоритм

In [8]:
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0] # кол-во неизвестных параметров
    T = len(V) # кол-во временных интервалов

    for n in range(n_iter):
        
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
        # в каждой итерации вычисляем alpha и beta 

        xi = np.zeros((M, M, T - 1)) # 3 мерная матрица со сторонами i, j и T
        for t in range(T - 1):
            denominator = (alpha[t, :].T @ a * b[:, V[t + 1]].T) @ beta[t + 1, :]
            # Берем скалярное произведение между alpha во время t и вероятностями перехода (матрицей a),
            # которое умножаем на вероятности выделения для наблюдения O во время t, и, берем скалярное произведение на beta
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T # Вычисление делителя для ξᵢⱼ(t)
                xi[i, :, t] = numerator / denominator #  деление числителя на знаменатель 
                

        gamma = np.sum(xi, axis=1) # Вычислим истинные значения gamma
        
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1)) # Делаем шаг максимилизации

        # Добавляем дополнительный T элемент
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1)))) # Добавлем последний элемент gamma
        # еще раз, в качестве последнего
        K = b.shape[1] # Кол-во уникальных наблюдений
        denominator = np.sum(gamma, axis=1) # Вычисляем знаменатель, который включает суммирование gamma по размерности 1
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1) # Вычисляем bⱼₖ, перебирая уникальные наблюдаемые значения

        b = np.divide(b, denominator.reshape((-1, 1))) # Последний шаг - разделение числитель на знаменатель

    return a, b